In [17]:
#https://machinelearningmastery.com/develop-first-xgboost-model-python-scikit-learn/
import bz2
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import random
import numpy as np
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm
from tensorflow import keras
import pandas as pd
import gensim
from collections import Counter
from datetime import datetime
from xgboost import XGBClassifier
from sklearn import tree
from tensorflow.keras.preprocessing.sequence import pad_sequences
PALABRAS = 100000
NUM_WORDS=100

In [2]:
silk_items = pd.read_csv("silkroad_items.csv")

In [3]:
boundary = silk_items["timestamp"][int((len(silk_items)-1)*0.8)]

In [4]:
categorias = list(silk_items["category"].unique())
dicc={}
for i in range(len(categorias)):
    dicc[categorias[i]] = i
dicc["misc"] = len(categorias)+1

In [5]:
silk_items["category"] = silk_items["category"].map(lambda x: dicc[x])

In [6]:

silk_items_train = silk_items[silk_items["timestamp"] <= boundary].filter(["title","description", "category"])
silk_items_train_y = silk_items_train["category"]
del silk_items_train["category"]


silk_items_validation = silk_items[silk_items["timestamp"] > boundary].filter(["title","description", "category"])
silk_items_validation_y = silk_items_validation["category"]
del silk_items_validation["category"]


In [7]:
w2v_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

In [8]:
silk_items_train["titulos + descripcion"] = (silk_items_train["title"] + silk_items_train["description"]).str.lower().map(lambda x: nltk.word_tokenize(x))
silk_items_train_test = silk_items_train.filter(["titulos + descripcion"])

In [9]:
emb_matrix = np.concatenate([np.zeros((1,300)), 
                             np.random.normal(size=(1,300)),
                             w2v_model.vectors[:PALABRAS]], axis=0)

In [10]:
textos_tokenizados = list(silk_items_train_test["titulos + descripcion"])

In [11]:
MAX_LEN = int(np.percentile([len(t) for t in textos_tokenizados], 95))

In [12]:
def preprocess_toks(textos_tokenizados):
    textos_tokenizados =textos_tokenizados.copy()
    if len(textos_tokenizados) > MAX_LEN:
        textos_tokenizados = textos_tokenizados[:MAX_LEN]
    for i in range(len(textos_tokenizados)):
        if textos_tokenizados[i] in w2v_model and w2v_model.key_to_index[textos_tokenizados[i]]<PALABRAS:
            textos_tokenizados[i] = w2v_model.key_to_index[textos_tokenizados[i]]+2
        else:
            textos_tokenizados[i] = 1
    return textos_tokenizados

In [13]:
for i in tqdm(range(len(textos_tokenizados)), total=len(textos_tokenizados)):
    textos_tokenizados[i] = preprocess_toks(textos_tokenizados[i])

100%|████████████████████████████████| 320875/320875 [00:23<00:00, 13576.04it/s]


In [14]:
w2v_model = None
silk_items_train = None
silk_items_train_test = None

In [15]:
emb_df = pd.DataFrame(textos_tokenizados).fillna(1)

In [18]:
emb_seq = pad_sequences(
    textos_tokenizados, maxlen=MAX_LEN, dtype='int32', padding='post',
    truncating='post', value=0
)


In [19]:
textos_tokenizados = None

In [20]:
clf = tree.DecisionTreeClassifier()

In [21]:
clf.fit(emb_seq, silk_items_train_y)

DecisionTreeClassifier()

In [22]:
silk_items_validation["titulos + descripcion"] = (silk_items_validation["title"] + silk_items_validation["description"]).str.lower().map(lambda x: nltk.word_tokenize(x))
silk_items_validation_test = list(silk_items_validation["titulos + descripcion"])

In [23]:
w2v_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

In [24]:
for i in tqdm(range(len(silk_items_validation_test)), total=len(silk_items_validation_test)):
    silk_items_validation_test[i] = preprocess_toks(silk_items_validation_test[i])

100%|███████████████████████████████████| 80219/80219 [00:09<00:00, 8658.88it/s]


In [25]:
emb_df_val = pd.DataFrame(silk_items_validation_test).fillna(1)

In [26]:
emb_val_seq = pad_sequences(
    silk_items_validation_test, maxlen=MAX_LEN, dtype='int32', padding='post',
    truncating='post', value=0
)


In [27]:
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, ConfusionMatrixDisplay, confusion_matrix

In [28]:
accuracy_score(silk_items_validation_y, clf.predict(emb_val_seq))

0.8723743751480323

In [29]:
versus = pd.read_csv("versus.csv")

In [30]:
versus_y = versus["Category"].map(lambda x: x if x in dicc.keys() else "misc" ).map(lambda x: dicc[x])

In [31]:
versus_val = list(versus["title"].str.lower().map(lambda x: nltk.word_tokenize(x)))

In [32]:
for i in tqdm(range(len(versus_val)), total=len(versus_val)):
    versus_val[i] = preprocess_toks(versus_val[i])

100%|█████████████████████████████████| 16015/16015 [00:00<00:00, 227562.09it/s]


In [33]:
emb_versus_val = pd.DataFrame(versus_val).fillna(1)

In [34]:
emb_versus_seq = pad_sequences(
    versus_val, maxlen=MAX_LEN, dtype='int32', padding='post',
    truncating='post', value=0
)


In [37]:
accuracy_score(versus_y, clf.predict(emb_versus_seq))

0.0333437402435217